In [ ]:
import pandas as pd
import torch
from transformers import pipeline

In [ ]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv("bld/python/data/data_clean.csv")

# Current Approach

In [ ]:
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
model_name = "facebook/bart-large-mnli"

In [ ]:
pipe = pipeline(model=model_name)

In [ ]:
def process_row(row):
    return pipe(row, candidate_labels=candidate_labels)

In [ ]:
df = df[:10]

In [ ]:
df["Classification"] = df["Article text"].apply(process_row)

In [ ]:
df["Classification"][3]

# (not useful) handle batches of data

In [ ]:
df

In [ ]:
batch_size = 16  # Set your desired batch size

# Calculate the number of batches needed
num_batches = (len(df) + batch_size - 1) // batch_size

# Split the DataFrame into batches and process each batch
for batch_idx in range(num_batches):
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(df))

    batch_data = df.iloc[batch_start:batch_end]

In [ ]:
batch_data

# put to tensors

In [ ]:
import torch

In [ ]:
text_column = df["Article text"]

In [ ]:
text_list = text_column.tolist()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [ ]:
tokenized_batch = tokenizer(
    text_list,
    padding=True,
    truncation=True,
    return_tensors="pt",
)

In [ ]:
input_ids = tokenized_batch["input_ids"]
attention_mask = tokenized_batch["attention_mask"]

In [ ]:
input_ids_tensor = input_ids.clone().detach()
attention_mask_tensor = attention_mask.clone().detach()

In [ ]:
attention_mask_tensor

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name,
    tokenizer=tokenizer,
    multi_label=True,
)

In [ ]:
result = classifier(text_list, candidate_labels, attention_mask=attention_mask)

In [ ]:
print(result)

# New Approach

# Best approach

Note: Lecture 7 is key to my problem

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, pipeline

In [ ]:
df = load_from_disk("bld/python/data/data_clean")
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
model_name = "facebook/bart-large-mnli"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["Article text"], padding="max_length", truncation=True)

In [ ]:
df_encoded = df.map(tokenize, batched=True, batch_size=None)

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
sequence_to_classify = (
    "Tiger Woods: Is this the end of his era? - CNN,Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. ",
    "golf, Tiger Woods: Is this the end of his era? - CNN,Is this the end of the Tiger Woods era?,This story was excerpted from the November 23 edition of CNN's Meanwhile in America, the daily email about US politics for global readers. Click here to read past editions and subscribe. (CNN)Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. Woods, who is recuperating from devastating leg injuries from a car crash, told Golf Digest he would have to be more selective about competition from now on. "
    "I think something that is realistic",
)

In [ ]:
classifier(sequence_to_classify, candidate_labels, tokenizer=tokenizer)

## Functionize it

#### Reasoning for new model

https://huggingface.co/valhalla/distilbart-mnli-12-1 has 90% of the facebook/bart-large-mnli model's performance but is way faster

In [ ]:
import pandas as pd
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, pipeline

In [ ]:
df = load_from_disk("bld/python/data/data_clean")
model_name = "facebook/bart-large-mnli"

In [ ]:
model_name_2 = "valhalla/distilbart-mnli-12-1"

In [ ]:
from transformers import AutoTokenizer


def zero_shot_labelling(data):
    model_name = "valhalla/distilbart-mnli-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16,  # adjust batch size
    )


# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"


def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding=True, truncation=True, max_length=42)


# Call zero_shot_labelling function

In [ ]:
# automodel = AutoModelForSequenceClassification.from_pretrained(model_name)
from transformers import AutoTokenizer, AutoModel, , AutoModelForSequenceClassification


def zero_shot_labelling(data):
    model_name = "facebook/bart-large-mnli"
    tokenizer = AutoModelForSequenceClassification.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16, # adjust batch size
    )
# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"

def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding="max_length", truncation=True)


# Call zero_shot_labelling function
df_encoded = zero_shot_labelling(df)

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name_2,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
sequence_to_classify = (
    "Tiger Woods: Is this the end of his era? - CNN,Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. ",
    "golf, Tiger Woods: Is this the end of his era? - CNN,Is this the end of the Tiger Woods era?,This story was excerpted from the November 23 edition of CNN's Meanwhile in America, the daily email about US politics for global readers. Click here to read past editions and subscribe. (CNN)Tiger Woods is the rarest of athletes. At his peak, he transcended his sport. People who couldn't care less about golf watched in their millions on Sunday afternoons to see him roar. So the 15-time major champ's announcement that he is calling time on life as a full-time pro feels like the end of an era. Woods, who is recuperating from devastating leg injuries from a car crash, told Golf Digest he would have to be more selective about competition from now on. "
    "I think something that is realistic",
)

In [ ]:
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
from datasets import Dataset, DatasetDict

df_try = df


def pd_to_dataset(data):
    data = Dataset.from_pandas(data)
    dataset_dict = DatasetDict({"my_dataset": data})
    return dataset_dict["my_dataset"]


df_encoded = zero_shot_labelling(df_try)

In [ ]:
df_encoded

In [ ]:
classifier(df_encoded["Description"], candidate_labels, tokenizer=_tokenize)

need to speed it up:
- Batch size of 8
- padding can be reduced to speed up computation
- 

## Approach to be faster

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

nli_model = AutoModelForSequenceClassification.from_pretrained(
    "facebook/bart-large-mnli",
)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [ ]:
df_encoded = [{"Article text": str(item["Article text"])} for item in df_encoded]

In [ ]:
x = tokenizer.encode(df_encoded["Article text"],, return_tensors='pt',
                     truncation_strategy='only_first')

In [ ]:
from transformers import AutoTokenizer


def zero_shot_labelling(data):
    model_name = "valhalla/distilbart-mnli-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16,  # adjust batch size
    )


# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"


def _tokenize(batch, tokenizer):
    return tokenizer(batch["Article text"], padding="max_length", truncation=True)


# Call zero_shot_labelling function
df_encoded = zero_shot_labelling(df)

## free up space every time before running the code

In [ ]:
from huggingface_hub import scan_cache_dir

delete_strategy = scan_cache_dir().delete_revisions(
    "81fd1d6e7847c99f5862c9fb81387956d99ec7aa"
    "e2983b237dccf3ab4937c97fa717319a9ca1a96d",
    "6c0e6080953db56375760c0471a8c5f2929baf11",
)
print("Will free " + delete_strategy.expected_freed_size_str)


delete_strategy.execute()

# Specify the directory you want to clear the cache for
cache_directory = "/path/to/your/cache/directory"

# Use scan_cache_dir to clear the cache in the specified directory
scan_cache_dir(cache_directory).clear()

## More cleaning

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("bld/python/data/data_clean.csv")

In [ ]:
aList = df

In [ ]:
import csv

with open(self.cleaned_data.csv) as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_NONE)
    for row in reader:
        yield (cell.strip() for cell in row)

In [ ]:
df = df.dropna(how="all")

In [ ]:
df.to_csv("cleaned_data.csv", index=False)

In [ ]:
df

In [ ]:
df = df[df.apply(lambda row: len(row[0].split(",")) == 4, axis=1)]

## Internet Notepad Approach

In [ ]:
from datasets import Dataset, DatasetDict, load_from_disk

df = load_from_disk("bld/python/data/data_clean")

In [ ]:
df

In [ ]:
df_csv = pd.read_csv("bld/python/data/data_clean.csv")

In [ ]:
import torch
from torch.utils.data.dataset import Dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # choose model

In [ ]:
df_csv = df_csv[["Article text", "Category"]]
df_csv["Category"] = df_csv["Category"].astype("category")
df_csv["text"] = df_csv["Article text"]
df_csv = df_csv.drop("Article text", axis=1)

In [ ]:
df_csv["Category_code"] = df_csv.Category
df_csv["Category_code"] = df_csv.Category.cat.codes

In [ ]:
df_csv["label"] = df_csv["Category_code"]
df_csv = df_csv.drop("Category_code", axis=1)
df_csv.head()

In [ ]:
max_length = 512


def tokenize_data_2(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )

In [ ]:
df_csv = df_csv.map(tokenize_data_2, batched=True)

# Neuer try

In [ ]:
import pandas as pd
import torch

In [ ]:
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, pipeline

df = load_from_disk("bld/python/data/data_clean")

In [ ]:
df = zero_shot_labelling(df)

In [ ]:
from transformers import AutoTokenizer


def zero_shot_labelling(data):
    model_name = "valhalla/distilbart-mnli-12-1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=16,  # adjust batch size
    )


# batch of 8: 47.4, padding = True
# batch of 16: 41.3, padding True
# batch of 16: 38.4, padding = "max_length"


def _tokenize(batch, tokenizer):
    return tokenizer(batch["Description"], padding=True, truncation=True, max_length=42)


# Call zero_shot_labelling function

In [ ]:
candidate_labels = ["labor supply", "labor demand", "government intervention"]

In [ ]:
model_name_2 = "valhalla/distilbart-mnli-12-1"

In [ ]:
classifier = pipeline(
    "zero-shot-classification",
    model=model_name_2,
    multi_label=True,
    device="cuda:0" if torch.cuda.is_available() else None,
)

In [ ]:
classifier(df["Description"], candidate_labels, tokenizer=_tokenize)

In [ ]:
df

## just functions

In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer, pipeline

df = load_from_disk("bld/python/data/data_clean")

In [ ]:
def select_random_entries(dataframe, num_entries=50, random_state=42):
    """Select a random set of entries from a Pandas DataFrame.

    Parameters:
    - dataframe (pd.DataFrame): The input DataFrame with 6 columns.
    - num_entries (int): The number of random entries to select (default is 50).
    - random_state (int or None): Random seed for reproducibility (default is None).

    Returns:
    - pd.DataFrame: A DataFrame containing the randomly selected entries.
    """
    dataframe = pd.DataFrame(dataframe)
    # dataframe is json

    if random_state is not None:
        np.random.seed(random_state)  # Set the random seed

    # Check if num_entries is greater than the total number of rows
    if num_entries > len(dataframe):
        msg = "Number of entries to select cannot exceed the total number of rows."
        raise ValueError(
            msg,
        )

    # Use Pandas' sample method to select random entries
    return dataframe.sample(n=num_entries)

In [ ]:
select_random_entries(df, num_entries=50, random_state=42)

In [ ]:
pd.DataFrame(df)

In [ ]:
type(df)

In [ ]:
first_100_entries = df.select(range(100))

In [ ]:
class= zero_shot_classifier(first_100_entries)

In [ ]:
import random

import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, pipeline

seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


def zero_shot_classifier(data):
    """Classify the zero-shot data to receive the labels."""
    data = _zero_shot_labelling(data)
    model_name = "valhalla/distilbart-mnli-12-6"
    labels = ["labor supply", "labor demand", "government intervention"]
    classifier = pipeline(  # second last
        "zero-shot-classification",
        model=model_name,
        multi_label=True,
        device="cuda:0" if torch.cuda.is_available() else None,
    )
    return classifier(  # last
        data["Description"],
        labels,
        tokenizer=_tokenize,
    )


def _zero_shot_labelling(data):
    """Load the model for zero-shot classification and apply on the data."""
    model_name = "valhalla/distilbart-mnli-12-6"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=8,
    )


def _tokenize(batch, tokenizer):
    """Define the tokenizer."""
    return tokenizer(
        batch["Description"],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

# For reading to know

In [ ]:
# run the previous
select_random_entries(df, num_entries=50, random_state=42)

# Training the model

TODO:
- does the probability of the model suits or should I transform to 0 and 1
- test and training separation
- model selection
- put the head on it

In [ ]:
import json

import pandas as pd
import torch
from transformers import AutoTokenizer, pipeline

In [ ]:
file_path = "bld\\python\\labelled\\data_labelled_subset.json"

df = pd.read_json(file_path)

In [ ]:
df

In [ ]:
# split the data
# better way with arrays
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["sequence"],
    df["scores"],
    test_size=0.15,
    random_state=8,
)

In [ ]:
adadd = load_from_disk("bld/python/labelled/data_labelled_subset.json")

In [ ]:
from datasets import load_from_disk

data = load_from_disk("bld/python/data/data_clean")
first_100_entries = data.select(range(100))

In [ ]:
df = zero_shot_classifier(first_100_entries)

In [ ]:
zero_shot_labelling(first_100_entries)

In [ ]:
def zero_shot_labelling(data):
    """Load the model for zero-shot classification and apply on the data."""
    model_name = "valhalla/distilbart-mnli-12-6"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    df_encoded = data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=8,
    )
    df_encoded.set_format(
        "torch",
        columns=["input_ids", "attention_mask"],
    )
    df_encoded.set_format("torch")
    return df_encoded


def _tokenize(batch, tokenizer):
    """Define the tokenizer."""
    return tokenizer(
        batch["Description"],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

In [ ]:
"""Functions for fitting the regression model."""

import random

import torch
from transformers import AutoTokenizer, pipeline


def _zero_shot_labelling(data):
    """Load the model for zero-shot classification and apply on the data."""
    model_name = "valhalla/distilbart-mnli-12-6"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    df_encoded = data.map(
        lambda batch: _tokenize(batch, tokenizer),
        batched=True,
        batch_size=8,
    )
    df_encoded.set_format(
        "torch",
        columns=["input_ids", "attention_mask"],
    )
    df_encoded.set_format("torch")
    return df_encoded


def _tokenize(batch, tokenizer):
    """Define the tokenizer."""
    return tokenizer(
        batch["Description"],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

In [ ]:
import json

# Specify the file path
file_path = "bld\\python\\labelled\\data_labelled_subset.json"

# Open and load the JSON file
with open(file_path) as json_file:
    df = json.load(json_file)

In [ ]:
# split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["sequence"],
    df["scores"],
    test_size=0.15,
    random_state=8,
)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
max_length = 512


def tokenize_data(df):
    return tokenizer(
        df["sequence"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
    )

In [ ]:
df = df.map(tokenize_data, batched=True)